### このノートブックは，[Understanding the Evaluation Metric ](https://www.kaggle.com/code/debarshichanda/understanding-mean-average-precision)を日本語にしたものです．

### H&Mコンペの評価尺度である，MAPの理解になればと思います．  

##### ※分かりやすくするために，多少表現を変えたり，追記している箇所があります．

----

## 評価尺度を理解する
このノートブックは，[this great notebook](https://www.kaggle.com/pestipeti/explanation-of-map5-scoring-metric)を参考にしています．

# Mean Average Precision(MAP)

Submissionは，Mean Average Precision@12（MAP@12）で評価されます．

$$MAP@12 = {1 \over U} \sum_{u=1}^{U} \sum_{k=1}^{min(n,12)}P(k) \times rel(k)$$

`U`はユーザーの数，`P(k)`は，k位までの精度です．`rel(k)`は`k`位が正しければ1を，誤っていれば０を返す関数です．また，nは予測数です（今回の場合は推薦する商品数）．

最終的なMAP算出関数に向けて，一歩ずつ組み立ていきましょう．

In [ ]:
import numpy as np

# 真の値（ground_truth）
gt = np.array(['a', 'b', 'c', 'd', 'e'])

# 予測値
preds1 = np.array(['b', 'c', 'a', 'd', 'e'])
preds2 = np.array(['a', 'b', 'c', 'd', 'e'])
preds3 = np.array(['f', 'b', 'c', 'd', 'e'])
preds4 = np.array(['a', 'f', 'e', 'g', 'b'])
preds5 = np.array(['a', 'f', 'c', 'g', 'b'])
preds6 = np.array(['d', 'c', 'b', 'a', 'e'])

# Precision

Precisionは，真陽性率のことです．分類問題の際には，モデルから算出されるtotalの陽性率を，真陽性数で割ることで得られます．

$$ P = { \#\ of\ correct\ predictions\over \#\ of\ all\ predictions  } = {TP \over (TP + FP)}$$

<br>

情報検索の分野では，Precisionは適合した文書の割合を表します．

$${\displaystyle {\text{P}}={\frac {|\{{\text{relevant documents}}\}\cap \{{\text{retrieved documents}}\}|}{|\{{\text{retrieved documents}}\}|}}}$$

# Precision at K

k位で区切った精度である，`P(k)`は，1位からk位までの集合を考えて求めることができる精度です．  
この値を計算することで，k位までのレコメンデーションを取ることができ，正解データをもとにした精度を考えることができます． <br>
<br>
Example 1:
`gt=[a,b,c,d,e]` と `pred=[b,c,a,d,e]` について`P@1` を考えます．`pred`からの1つのレコメンデーションとして`b`を取り，`gt`との`precision`を計算すると以下のようになります．<br>
<br>
$${\displaystyle {\text{P}}={\frac {|\{{\text{gt}}\}\cap \{{\text{pred[:1]}}\}|}{|\{{\text{pred[:1]}}\}|}}={\frac {\text{1}}{\text{1}}}}$$ 
<br>
Example 2:
`gt=[a,b,c,d,e]` と `pred=[f,b,c,d,e]` について`P@1`　を考えます．`pred`からの1つのレコメンデーションとして，`f`を取り，`gt`との`precision`を計算すると以下のようになります．<br>
<br>
$${\displaystyle {\text{P}}={\frac {|\{{\text{gt}}\}\cap \{{\text{pred[:1]}}\}|}{|\{{\text{pred[:1]}}\}|}}={\frac {\text{0}}{\text{1}}}}$$
<br>
Example 3:
`gt=[a,b,c,d,e]` と `pred=[a,f,e,g,b]` について`P@2` を考えます．`pred`からの2つのレコメンデーションとして，`[a,f]`を取り，`gt`との`precision`を計算します．`gt`に現れるのは`a`たった１つであるため，共通集合数は`１`になります．<br>
<br>
$${\displaystyle {\text{P}}={\frac {|\{{\text{gt}}\}\cap \{{\text{pred[:2]}}\}|}{|\{{\text{pred[:2]}}\}|}}={\frac {\text{1}}{\text{2}}}}$$
<br>

具体例は以下のようになります．

| true  | predicted   | k  | P(k) |
|:-:|:-:|:-:|:-:|
| [a, b, c, d, e]  | [b, c, a, d, e]   | 1  | 1.0  |
| [a, b, c, d, e]  | [a, b, c, d, e]   | 1  | 1.0  |
| [a, b, c, d, e]  | [f, b, c, d, e]   | 1  | 0.0  |
| [a, b, c, d, e]  | [a, f, e, g, b]   | 2  | $$1\over2$$  |
| [a, b, c, d, e]  | [a, f, c, g, b]   | 3  | $$2\over3$$  |
| [a, b, c, d, e]  | [d, c, b, a, e]   | 3  | $$3\over3$$  |

In [ ]:
def precision_at_k(y_true, y_pred, k=12):
    """ Computes Precision at k for one sample
    
    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           Precision at k
    """
    intersection = np.intersect1d(y_true, y_pred[:k])
    return len(intersection) / k

In [ ]:
assert precision_at_k(gt, preds1, k=1) == 1.0
assert precision_at_k(gt, preds2, k=1) == 1.0
assert precision_at_k(gt, preds3, k=1) == 0.0
assert precision_at_k(gt, preds4, k=2) == 1/2
assert precision_at_k(gt, preds5, k=3) == 2/3
assert precision_at_k(gt, preds6, k=3) == 3/3

# Rel at K

`Rel(k)` は，k位のアイテムが適合（正しい）場合に１を，そうでない場合は0を返します<br>
Example 1:
`gt=[a,b,c,d,e]` と `pred=[b,c,a,d,e]` について `rel@1` を考えます． `pred`からの1つのレコメンデーションとして，`b`を取り，　`gt`との`precision`を計算すると以下のようになります．<br>
$${\displaystyle {\text{rel(k)}}=1.0}$$
<br>
Example 2:
`gt=[a,b,c,d,e]` と `pred=[f,b,c,d,e]` について `rel@1` を考えます． `pred`からの１つのレコメンデーションとして，`f`を取り，`gt`との`precision`を計算すると以下のようになります．<br>
$${\displaystyle {\text{rel(k)}}=0.0}$$
<br>
Example 3:
`gt=[a,b,c,d,e]` と `pred=[a,f,e,g,b]` について `rel@2` を考えます． `pred`からの１つのレコメンデーションとして，`f`を取り，`gt`との`precision`を計算すると以下のようになります．
$${\displaystyle {\text{rel(k)}}=0.0}$$
<br>

具体例は以下のようになります．

| true  | predicted   | k  | rel(k) |
|:-:|:-:|:-:|:-:|
| [a, b, c, d, e]  | [b, c, a, d, e]   | 1  | 1.0  |
| [a, b, c, d, e]  | [a, b, c, d, e]   | 1  | 1.0  |
| [a, b, c, d, e]  | [f, b, c, d, e]   | 1  | 0.0  |
| [a, b, c, d, e]  | [a, f, e, g, b]   | 2  | 0.0  |
| [a, b, c, d, e]  | [a, f, c, g, b]   | 3  | 1.0  |
| [a, b, c, d, e]  | [d, c, b, a, e]   | 3  | 1.0  |

In [ ]:
def rel_at_k(y_true, y_pred, k=12):
    """ Computes Relevance at k for one sample
    
    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           Relevance at k
    """
    if y_pred[k-1] in y_true:
        return 1
    else:
        return 0

In [ ]:
assert rel_at_k(gt, preds1, k=1) == 1.0
assert rel_at_k(gt, preds2, k=1) == 1.0
assert rel_at_k(gt, preds3, k=1) == 0.0
assert rel_at_k(gt, preds4, k=2) == 0.0
assert rel_at_k(gt, preds5, k=3) == 1.0
assert rel_at_k(gt, preds6, k=3) == 1.0

# Average Precision at K

全ての`k`についての`P@k`と`rel(k)`から，全製品について単純な平均を取ったものです．

$${1\over{{min(n,12)}}} {\sum_{k=1}^{min(n,12)}P(k) \times rel(k)}$$

In [ ]:
def average_precision_at_k(y_true, y_pred, k=12):
    """ Computes Average Precision at k for one sample
    
    Parameters
    __________
    y_true: np.array
            Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           Average Precision at k
    """
    ap = 0.0
    for i in range(1, k+1):
        ap += precision_at_k(y_true, y_pred, i) * rel_at_k(y_true, y_pred, i)
        
    return ap / min(k, len(y_true))

In [ ]:
assert average_precision_at_k(gt, preds1, k=1) == 1.0
assert average_precision_at_k(gt, preds2, k=1) == 1.0
assert average_precision_at_k(gt, preds3, k=1) == 0.0
assert average_precision_at_k(gt, preds4, k=2) == 0.5
assert average_precision_at_k(gt, preds5, k=3) == 0.5555555555555555
assert average_precision_at_k(gt, preds6, k=3) == 1.0

# Mean Average Precision at K

全ユーザーについてAP（Average Precision）の平均を取ったものです．

In [ ]:
def mean_average_precision(y_true, y_pred, k=12):
    """ Computes MAP at k
    
    Parameters
    __________
    y_true: np.array
            2D Array of correct recommendations (Order doesn't matter)
    y_pred: np.array
            2D Array of predicted recommendations (Order does matter)
    k: int, optional
       Maximum number of predicted recommendations
            
    Returns
    _______
    score: double
           MAP at k
    """
    return np.mean([average_precision_at_k(gt, pred, k) \
                    for gt, pred in zip(y_true, y_pred)])

In [ ]:
y_true = np.array([gt, gt, gt, gt, gt, gt])
y_pred = np.array([preds1, preds2, preds3, preds4, preds5, preds6])

In [ ]:
print(average_precision_at_k(gt, preds1, k=4))
print(average_precision_at_k(gt, preds2, k=4))
print(average_precision_at_k(gt, preds3, k=4))
print(average_precision_at_k(gt, preds4, k=4))
print(average_precision_at_k(gt, preds5, k=4))
print(average_precision_at_k(gt, preds6, k=4))

In [ ]:
mean_average_precision(y_true, y_pred, k=4)

### 読んで頂きありがとうございました．